# Introduction

Without using transfer learning, goal is to train models with and without using the metadata. ALso let's try implementing a 1-cycle learning rate schedule.

In [1]:
# imports
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import seaborn as sns  # for nicer plots
sns.set(style="darkgrid")  # default style
import os

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score

# baseline classifier
from sklearn.dummy import DummyRegressor

# random forest
from sklearn.ensemble import RandomForestRegressor

import tensorflow as tf

gpu = len(tf.config.list_physical_devices('GPU'))>0
print("GPU is", "available" if gpu else "NOT AVAILABLE")

GPU is available


In [2]:
# load metadata
train_images = os.listdir('petfinder-pawpularity-score/train')
train_df = pd.read_csv('petfinder-pawpularity-score/train.csv')
train_df = train_df.set_index('Id')

In [3]:
train_df

Subject Focus  Eyes  Face  Near  Action  \
Id                                                                          
0007de18844b0dbbb5e1f607da0606e0              0     1     1     1       0   
0009c66b9439883ba2750fb825e1d7db              0     1     1     0       0   
0013fd999caf9a3efe1352ca1b0d937e              0     1     1     1       0   
0018df346ac9c1d8413cfcc888ca8246              0     1     1     1       0   
001dc955e10590d3ca4673f034feeef2              0     0     0     1       0   
...                                         ...   ...   ...   ...     ...   
ffbfa0383c34dc513c95560d6e1fdb57              0     0     0     1       0   
ffcc8532d76436fc79e50eb2e5238e45              0     1     1     1       0   
ffdf2e8673a1da6fb80342fa3b119a20              0     1     1     1       0   
fff19e2ce11718548fa1c5d039a5192a              0     1     1     1       0   
fff8e47c766799c9e12f3cb3d66ad228              0     1     1     1       0   

                                  Accessory  Group  Collage  Human  Occlusion  \
Id                                                                              
0007de18844b0dbbb5e1f607da0606e0          0      1        0      0          0   
0009c66b9439883ba2750fb825e1d7db          0      0        0      0          0   
0013fd999caf9a3efe1352ca1b0d937e          0      0        0      1          1   
0018df346ac9c1d8413cfcc888ca8246          0      0        0      0          0   
001dc955e10590d3ca4673f034feeef2          0      1        0      0          0   
...                                     ...    ...      ...    ...        ...   
ffbfa0383c34dc513c95560d6e1fdb57          0      0        0      0          0   
ffcc8532d76436fc79e50eb2e5238e45          0      0        0      0          0   
ffdf2e8673a1da6fb80342fa3b119a20          0      0        0      1          1   
fff19e2ce11718548fa1c5d039a5192a          0      0        0      1          0   
fff8e47c766799c9e12f3cb3d66ad228          0      0        0      0          0   

                                  Info  Blur  Pawpularity  
Id                                                         
0007de18844b0dbbb5e1f607da0606e0     0     0           63  
0009c66b9439883ba2750fb825e1d7db     0     0           42  
0013fd999caf9a3efe1352ca1b0d937e     0     0           28  
0018df346ac9c1d8413cfcc888ca8246     0     0           15  
001dc955e10590d3ca4673f034feeef2     0     0           72  
...                                ...   ...          ...  
ffbfa0383c34dc513c95560d6e1fdb57     0     1           15  
ffcc8532d76436fc79e50eb2e5238e45     0     0           70  
ffdf2e8673a1da6fb80342fa3b119a20     0     0           20  
fff19e2ce11718548fa1c5d039a5192a     0     0           20  
fff8e47c766799c9e12f3cb3d66ad228     0     0           30  

[9912 rows x 13 columns]

# dataset objects

In [9]:
train_images = os.listdir('petfinder-pawpularity-score/train')
train_df = pd.read_csv('petfinder-pawpularity-score/train.csv')
train_df = train_df.set_index('Id')

file_names = []
labels = []

for index in train_df.index:
    file_name = 'petfinder-pawpularity-score/train/' + index + '.jpg'
    label = train_df.loc[index]['Pawpularity']
    file_names.append(file_name)
    labels.append(label/100.01)
    #labels.append(label*1.0)

train_files, val_files, labels_train, labels_val = train_test_split(file_names, labels, test_size=0.2, random_state=137)

def process_file_names_train(file_name, label):
    image_string = tf.io.read_file(file_name)
    img = tf.io.decode_jpeg(image_string, channels=3)
    img = tf.cast(img, tf.float32)
    img = tf.image.resize(img, [224,224])
    img = tf.keras.applications.efficientnet.preprocess_input(img) 
    # drop this for val data
    img = tf.image.random_flip_left_right(img)
    img = tf.image.random_saturation(img, 0.95, 1.05)
    img = tf.image.random_contrast(img, 0.95, 1.05)
    

    #img = tf.image.random_hue(img, 0.05)
    #img = tf.image.random_saturation(img, 0.7, 1.3)
    #img = tf.image.random_contrast(img, 0.8, 1.2)
    #img = tf.image.random_brightness(img, 0.15)
    return img/255., label

def process_file_names_val(file_name, label):
    image_string = tf.io.read_file(file_name)
    img = tf.io.decode_jpeg(image_string, channels=3)
    img = tf.cast(img, tf.float32)
    img = tf.image.resize(img, [224,224])
    img = tf.keras.applications.efficientnet.preprocess_input(img) 
    return img/255., label

train_dataset = tf.data.Dataset.from_tensor_slices((train_files, labels_train))
train_dataset = train_dataset.map(process_file_names_train)

val_dataset = tf.data.Dataset.from_tensor_slices((val_files, labels_val))
val_dataset = val_dataset.map(process_file_names_val)

def configure_for_performance(ds):
    ds = ds.cache()
    ds = ds.shuffle(buffer_size=1000)
    ds = ds.batch(32)
    return ds

train_dataset = configure_for_performance(train_dataset)

val_dataset = configure_for_performance(val_dataset)

# VGGNet

In [5]:
def vgg16(input_shape=(224, 224, 3)):
    tf.keras.backend.clear_session()
    model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=input_shape),
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
        tf.keras.layers.MaxPooling2D((2, 2), strides=(2, 2)),
        
        tf.keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
        tf.keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
        tf.keras.layers.MaxPooling2D((2, 2), strides=(2, 2)),
        
        tf.keras.layers.Conv2D(256, (3, 3), activation='relu', padding='same'),
        tf.keras.layers.Conv2D(256, (3, 3), activation='relu', padding='same'),
        tf.keras.layers.Conv2D(256, (3, 3), activation='relu', padding='same'),
        tf.keras.layers.MaxPooling2D((2, 2), strides=(2, 2)),
        
        tf.keras.layers.Conv2D(512, (3, 3), activation='relu', padding='same'),
        tf.keras.layers.Conv2D(512, (3, 3), activation='relu', padding='same'),
        tf.keras.layers.Conv2D(512, (3, 3), activation='relu', padding='same'),
        tf.keras.layers.MaxPooling2D((2, 2), strides=(2, 2)),
        
        tf.keras.layers.Conv2D(512, (3, 3), activation='relu', padding='same'),
        tf.keras.layers.Conv2D(512, (3, 3), activation='relu', padding='same'),
        tf.keras.layers.Conv2D(512, (3, 3), activation='relu', padding='same'),
        tf.keras.layers.MaxPooling2D((2, 2), strides=(2, 2)),
        
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(4096, activation='relu'),
        tf.keras.layers.Dense(4096, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')
        #tf.keras.layers.Dense(num_classes, activation='softmax')
    ])
    
    return model

# Create VGG16 model
model = vgg16()

# Display model summary
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 224, 224, 64)      1792      
                                                                 
 conv2d_1 (Conv2D)           (None, 224, 224, 64)      36928     
                                                                 
 max_pooling2d (MaxPooling2  (None, 112, 112, 64)      0         
 D)                                                              
                                                                 
 conv2d_2 (Conv2D)           (None, 112, 112, 128)     73856     
                                                                 
 conv2d_3 (Conv2D)           (None, 112, 112, 128)     147584    
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 56, 56, 128)       0         
 g2D)                                                   

In [6]:
early_stopping = tf.keras.callbacks.EarlyStopping(patience = 3,restore_best_weights=True)


lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-3,
    decay_steps=100, decay_rate=0.96,
    staircase=True)

In [7]:
model.compile(loss="mse", 
              optimizer = tf.keras.optimizers.legacy.Adam(learning_rate = lr_schedule), 
              metrics=[tf.keras.metrics.RootMeanSquaredError()])

# taking way too long to train

#predictor = model.fit(train_dataset,
#                      epochs=50, 
#                      validation_data = val_dataset,
#                      callbacks=[early_stopping])

Epoch 1/50


2024-04-07 20:31:13.167135: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


 5/62 [=>............................] - ETA: 9:47 - loss: 0.1549 - root_mean_squared_error: 0.3936

# Resnet

In [5]:
# define residual layer

class ResidualUnit(tf.keras.layers.Layer):
    def __init__(self, filters, strides=1, activation='relu', **kwargs):
        super().__init__(**kwargs)
        self.activation = tf.keras.activations.get(activation)
        self.main_layers = [
            tf.keras.layers.Conv2D(filters, kernel_size=3, strides=strides, padding='same', kernel_initializer='he_normal', use_bias=False),
            tf.keras.layers.BatchNormalization(),
            self.activation,
            tf.keras.layers.Conv2D(filters, kernel_size=3, strides=1, padding='same', kernel_initializer='he_normal', use_bias=False),
            tf.keras.layers.BatchNormalization()
        ]
        # If strides > 1 need to use 1D convolutions with long stride to resize image so they can be added
        if strides == 1:
            self.skip_layers = []
        else:
            self.skip_layers = [
                tf.keras.layers.Conv2D(filters, kernel_size=1, strides=strides, padding='same', kernel_initializer='he_normal', use_bias=False),
                tf.keras.layers.BatchNormalization()
            ]
    def call(self, inputs):
        z = inputs
        for layer in self.main_layers:
            z = layer(z)
        skip_z = inputs
        for layer in self.skip_layers:
            skip_z = layer(skip_z)
        # either return a(z) + skip_z or a(z + skip_z)
        return self.activation(z + skip_z)

In [6]:
def build_resnet():
    tf.keras.backend.clear_session()
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Conv2D(64, kernel_size=7, strides=2, padding='same', kernel_initializer='he_normal', use_bias=False, input_shape=[224,224,3]))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Activation('relu'))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=3, strides=2, padding='same'))
    prev_filters = 128
    for filters in [64]*3 + [128]*4 + [256]*6 + [512]*3:
        strides = 1 if filters == prev_filters else 2
        model.add(ResidualUnit(filters, strides=strides))
        prev_filters = filters


    model.add(tf.keras.layers.GlobalAvgPool2D())
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(units=1, activation="relu"))


    return model
    
model = build_resnet()

In [7]:
early_stopping = tf.keras.callbacks.EarlyStopping(patience = 3,restore_best_weights=True)


lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-3,
    decay_steps=100, decay_rate=0.96,
    staircase=True)

In [ ]:
early_stopping = tf.keras.callbacks.EarlyStopping(patience = 3,restore_best_weights=True)


lr_schedule = tf.keras.optimizers.schedules.CosineDecay(1e-5, 1000, warmup_target=3e-4, warmup_steps=100)

In [8]:
model.compile(loss="mse", 
              optimizer = tf.keras.optimizers.legacy.Adam(learning_rate = lr_schedule), 
              metrics=[tf.keras.metrics.RootMeanSquaredError()])


predictor = model.fit(train_dataset,
                      epochs=50, 
                      validation_data = val_dataset,
                      callbacks=[early_stopping])

Epoch 1/50


2024-04-07 20:38:55.498185: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


62/62 [==============================] - 74s 1s/step - loss: 257.9948 - root_mean_squared_error: 16.0622 - val_loss: 39021297664.0000 - val_root_mean_squared_error: 197538.0938
Epoch 2/50
62/62 [==============================] - 64s 1s/step - loss: 289.7820 - root_mean_squared_error: 17.0230 - val_loss: 7391590.5000 - val_root_mean_squared_error: 2718.7480
Epoch 3/50
62/62 [==============================] - 62s 996ms/step - loss: 290.8120 - root_mean_squared_error: 17.0532 - val_loss: 85155.2031 - val_root_mean_squared_error: 291.8137
Epoch 4/50
62/62 [==============================] - 62s 1s/step - loss: 291.0226 - root_mean_squared_error: 17.0594 - val_loss: 7595.6440 - val_root_mean_squared_error: 87.1530
Epoch 5/50
62/62 [==============================] - 61s 987ms/step - loss: 289.5814 - root_mean_squared_error: 17.0171 - val_loss: 1973.9180 - val_root_mean_squared_error: 44.4288
Epoch 6/50
62/62 [==============================] - 61s 986ms/step - loss: 288.9595 - root_mean_square

In [11]:
model = build_resnet()

early_stopping = tf.keras.callbacks.EarlyStopping(patience = 3,restore_best_weights=True)


lr_schedule = tf.keras.optimizers.schedules.CosineDecay(1e-5, 1000, warmup_target=3e-4, warmup_steps=100)

model.compile(loss="mse", 
              optimizer = tf.keras.optimizers.legacy.Adam(learning_rate = lr_schedule), 
              metrics=[tf.keras.metrics.RootMeanSquaredError()])


predictor = model.fit(train_dataset,
                      epochs=50, 
                      validation_data = val_dataset,
                      callbacks=[early_stopping])

Epoch 1/50
62/62 [==============================] - 76s 1s/step - loss: 1040.9561 - root_mean_squared_error: 32.2639 - val_loss: 3969394.2500 - val_root_mean_squared_error: 1992.3339
Epoch 2/50
62/62 [==============================] - 64s 1s/step - loss: 473.4985 - root_mean_squared_error: 21.7600 - val_loss: 1554.7184 - val_root_mean_squared_error: 39.4299
Epoch 3/50
62/62 [==============================] - 62s 995ms/step - loss: 403.6927 - root_mean_squared_error: 20.0921 - val_loss: 3503.7319 - val_root_mean_squared_error: 59.1923
Epoch 4/50
62/62 [==============================] - 63s 1s/step - loss: 368.0954 - root_mean_squared_error: 19.1858 - val_loss: 1223.2197 - val_root_mean_squared_error: 34.9746
Epoch 5/50
62/62 [==============================] - 64s 1s/step - loss: 294.5415 - root_mean_squared_error: 17.1622 - val_loss: 2749.2666 - val_root_mean_squared_error: 52.4334
Epoch 6/50
62/62 [==============================] - 63s 1s/step - loss: 169.8621 - root_mean_squared_error

In [10]:
def build_resnet():
    tf.keras.backend.clear_session()
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Conv2D(64, kernel_size=7, strides=2, padding='same', kernel_initializer='he_normal', use_bias=False, input_shape=[224,224,3]))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Activation('relu'))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=3, strides=2, padding='same'))
    prev_filters = 128
    for filters in [64]*3 + [128]*4 + [256]*6 + [512]*3:
        strides = 1 if filters == prev_filters else 2
        model.add(ResidualUnit(filters, strides=strides))
        prev_filters = filters


    model.add(tf.keras.layers.GlobalAvgPool2D())
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))


    return model

In [15]:
model = build_resnet()

early_stopping = tf.keras.callbacks.EarlyStopping(patience = 5,restore_best_weights=True)


lr_schedule = tf.keras.optimizers.schedules.CosineDecay(1e-7, 1000, warmup_target=3e-6, warmup_steps=500, alpha=1e-7)

model.compile(loss="binary_crossentropy", 
              optimizer = tf.keras.optimizers.legacy.Adam(learning_rate = lr_schedule), 
              metrics=[tf.keras.metrics.RootMeanSquaredError()])


predictor = model.fit(train_dataset,
                      epochs=50, 
                      validation_data = val_dataset,
                      callbacks=[early_stopping])

Epoch 1/50
248/248 [==============================] - 75s 281ms/step - loss: 0.6782 - root_mean_squared_error: 0.2181 - val_loss: 0.6708 - val_root_mean_squared_error: 0.2191
Epoch 2/50
248/248 [==============================] - 66s 265ms/step - loss: 0.6712 - root_mean_squared_error: 0.2117 - val_loss: 0.6758 - val_root_mean_squared_error: 0.2246
Epoch 3/50
248/248 [==============================] - 66s 267ms/step - loss: 0.6642 - root_mean_squared_error: 0.2040 - val_loss: 0.6786 - val_root_mean_squared_error: 0.2276
Epoch 4/50
248/248 [==============================] - 68s 273ms/step - loss: 0.6588 - root_mean_squared_error: 0.1977 - val_loss: 0.6771 - val_root_mean_squared_error: 0.2260
Epoch 5/50
248/248 [==============================] - 66s 267ms/step - loss: 0.6543 - root_mean_squared_error: 0.1924 - val_loss: 0.6756 - val_root_mean_squared_error: 0.2240
Epoch 6/50
248/248 [==============================] - 67s 269ms/step - loss: 0.6526 - root_mean_squared_error: 0.1903 - val_l

Let's just incorporate the metadata.

# create new dataset objects

In [4]:
for index in train_df.index:
    print(index)
    file_name = 'petfinder-pawpularity-score/train/' + index + '.jpg'

0007de18844b0dbbb5e1f607da0606e0
0009c66b9439883ba2750fb825e1d7db
0013fd999caf9a3efe1352ca1b0d937e
0018df346ac9c1d8413cfcc888ca8246
001dc955e10590d3ca4673f034feeef2
001dd4f6fafb890610b1635f967ea081
0023b8a3abc93c712edd6120867deb53
0031d6a9ef7340f898c3e05f92c7bb04
0042bc5bada6d1cf8951f8f9f0d399fa
0049cb81313c94fa007286e9039af910
005017716086b8d5e118dd9fe26459b1
00524dbf2637a80cbc80f70d3ff59616
00630b1262efe301cb15a3b2022ba744
006483b96ca9c09b7afed3e3d3af539d
00655425c10d4c082dd7eeb97fa4fb17
0067aaaa500b530c76b9c91af34b4cb8
006cda7fec46a527f9f627f4722a2304
006fe962f5f7e2c5f527b2e27e28ed6d
0075ec6503412f21cf65ac5f43d80440
00768659c1c90409f81dcdecbd270513
007c6ea0ccfca08d4736e5f459ed274a
00838b02764e8d37edde39ed18693427
0084a12f432ef6e47e8b9ff04d8a32b4
0085bfb9a7ebfd776cb804d8b456bb05
009367718f506d87cd898ed0310a2392
0095f81bab3b68a4f70e99f0fcec7b06
00a1ae8867e0bb89f061679e1cf29e80
00a1e0c0f89ff89a8f32d42e9025f6b2
00a6d367daed96f77be6c91db493dfed
00b151a572c9aabedf8cfce0fa18be25
00c7b59662

In [ ]:
images_train, images_test, metadata_train, metadata_test, y_train, y_test = train_test_split([1,2,3,4], [1,2,3,4], [1,2,3,4], test_size=0.5)

In [36]:
train_images = os.listdir('petfinder-pawpularity-score/train')
train_df = pd.read_csv('petfinder-pawpularity-score/train.csv')
train_df = train_df.set_index('Id')

file_names = []
labels = []
metadata = []

for index in train_df.index:
    file_name = 'petfinder-pawpularity-score/train/' + index + '.jpg'
    label = train_df.loc[index]['Pawpularity']
    metadata_row = train_df.loc[index][['Subject Focus', 'Eyes', 'Face', 'Near', 'Action', 'Accessory', 'Group',
       'Collage', 'Human', 'Occlusion', 'Info', 'Blur']].to_numpy()
    file_names.append(file_name)
    labels.append(label/100.01)
    metadata.append(metadata_row)
    #labels.append(label*1.0)

train_files, val_files, metadata_train, metadata_val, labels_train, labels_val = train_test_split(file_names, metadata, labels, test_size=0.2, random_state=137)

def process_file_names_train(file_name, metadata_row, label):
    image_string = tf.io.read_file(file_name)
    img = tf.io.decode_jpeg(image_string, channels=3)
    img = tf.cast(img, tf.float32)
    img = tf.image.resize(img, [224,224])
    img = tf.keras.applications.efficientnet.preprocess_input(img) 
    # drop this for val data
    img = tf.image.random_flip_left_right(img)
    img = tf.image.random_saturation(img, 0.95, 1.05)
    img = tf.image.random_contrast(img, 0.95, 1.05)
    

    #img = tf.image.random_hue(img, 0.05)
    #img = tf.image.random_saturation(img, 0.7, 1.3)
    #img = tf.image.random_contrast(img, 0.8, 1.2)
    #img = tf.image.random_brightness(img, 0.15)
    return img/255., metadata_row, label

def process_file_names_val(file_name, metadata_row, label):
    image_string = tf.io.read_file(file_name)
    img = tf.io.decode_jpeg(image_string, channels=3)
    img = tf.cast(img, tf.float32)
    img = tf.image.resize(img, [224,224])
    img = tf.keras.applications.efficientnet.preprocess_input(img) 
    return img/255., metadata_row, label


#combined_dataset = combined_dataset.map(lambda image, tabular, label: ({"image_input": image, "tabular_input": tabular}, label))

train_dataset = tf.data.Dataset.from_tensor_slices((train_files, metadata_train, labels_train))
train_dataset = train_dataset.map(process_file_names_train)
train_dataset = train_dataset.map(lambda image_lambda, metadata_lambda, label: ({'image_input': image_lambda, 'metadata_input': metadata_lambda}, label))

val_dataset = tf.data.Dataset.from_tensor_slices((val_files, metadata_val, labels_val))
val_dataset = val_dataset.map(process_file_names_val)
val_dataset = val_dataset.map(lambda image_lambda, metadata_lambda, label: ({'image_input': image_lambda, 'metadata_input': metadata_lambda}, label))

def configure_for_performance(ds):
    ds = ds.cache()
    ds = ds.shuffle(buffer_size=1000)
    ds = ds.batch(32)
    return ds

train_dataset = configure_for_performance(train_dataset)

val_dataset = configure_for_performance(val_dataset)

In [27]:
# define residual layer

class ResidualUnit(tf.keras.layers.Layer):
    def __init__(self, filters, strides=1, activation='relu', **kwargs):
        super().__init__(**kwargs)
        self.activation = tf.keras.activations.get(activation)
        self.main_layers = [
            tf.keras.layers.Conv2D(filters, kernel_size=3, strides=strides, padding='same', kernel_initializer='he_normal', use_bias=False),
            tf.keras.layers.BatchNormalization(),
            self.activation,
            tf.keras.layers.Conv2D(filters, kernel_size=3, strides=1, padding='same', kernel_initializer='he_normal', use_bias=False),
            tf.keras.layers.BatchNormalization()
        ]
        # If strides > 1 need to use 1D convolutions with long stride to resize image so they can be added
        if strides == 1:
            self.skip_layers = []
        else:
            self.skip_layers = [
                tf.keras.layers.Conv2D(filters, kernel_size=1, strides=strides, padding='same', kernel_initializer='he_normal', use_bias=False),
                tf.keras.layers.BatchNormalization()
            ]
    def call(self, inputs):
        z = inputs
        for layer in self.main_layers:
            z = layer(z)
        skip_z = inputs
        for layer in self.skip_layers:
            skip_z = layer(skip_z)
        # either return a(z) + skip_z or a(z + skip_z)
        return self.activation(z + skip_z)

In [38]:
def build_resnet_backbone():
    tf.keras.backend.clear_session()
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Conv2D(64, kernel_size=7, strides=2, padding='same', kernel_initializer='he_normal', use_bias=False, input_shape=[224,224,3]))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Activation('relu'))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=3, strides=2, padding='same'))
    prev_filters = 128
    for filters in [64]*3 + [128]*4 + [256]*6 + [512]*3:
        strides = 1 if filters == prev_filters else 2
        model.add(ResidualUnit(filters, strides=strides))
        prev_filters = filters


    model.add(tf.keras.layers.GlobalAvgPool2D())
    model.add(tf.keras.layers.Flatten())
    #model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))


    return model

image_backbone = build_resnet_backbone()

image_input = tf.keras.layers.Input(shape=[224,224,3], name='image_input')
metadata_input = tf.keras.layers.Input(shape=[12], name='metadata_input')

image_processed = image_backbone(image_input)

#metadata_embedded = tf.keras.layers.Embedding(12, 3)(metadata_input)

x = tf.keras.layers.concatenate([image_processed, metadata_input])

x = tf.keras.layers.Dense(32, activation='tanh')(x)

output = tf.keras.layers.Dense(units=1, activation="sigmoid")(x)

model = tf.keras.Model(inputs=[image_input, metadata_input], outputs=output)



In [33]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_input (InputLayer)    [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 sequential (Sequential)     (None, 512)                  2130604   ['image_input[0][0]']         
                                                          8                                       
                                                                                                  
 metadata_input (InputLayer  [(None, 12)]                 0         []                            
 )                                                                                                
                                                                                              

In [39]:
early_stopping = tf.keras.callbacks.EarlyStopping(patience = 5,restore_best_weights=True)


lr_schedule = tf.keras.optimizers.schedules.CosineDecay(1e-7, 1000, warmup_target=3e-6, warmup_steps=500, alpha=1e-7)

model.compile(loss="binary_crossentropy", 
              optimizer = tf.keras.optimizers.legacy.Adam(learning_rate = lr_schedule), 
              metrics=[tf.keras.metrics.RootMeanSquaredError()])


predictor = model.fit(train_dataset,
                      epochs=50, 
                      validation_data = val_dataset,
                      callbacks=[early_stopping])

Epoch 1/50


2024-04-10 13:48:27.150982: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


248/248 [==============================] - 90s 297ms/step - loss: 0.6802 - root_mean_squared_error: 0.2226 - val_loss: 0.6688 - val_root_mean_squared_error: 0.2171
Epoch 2/50
248/248 [==============================] - 68s 274ms/step - loss: 0.6672 - root_mean_squared_error: 0.2077 - val_loss: 0.6691 - val_root_mean_squared_error: 0.2170
Epoch 3/50
248/248 [==============================] - 67s 269ms/step - loss: 0.6632 - root_mean_squared_error: 0.2030 - val_loss: 0.6679 - val_root_mean_squared_error: 0.2159
Epoch 4/50
248/248 [==============================] - 68s 274ms/step - loss: 0.6602 - root_mean_squared_error: 0.1996 - val_loss: 0.6676 - val_root_mean_squared_error: 0.2157
Epoch 5/50
248/248 [==============================] - 67s 269ms/step - loss: 0.6573 - root_mean_squared_error: 0.1961 - val_loss: 0.6665 - val_root_mean_squared_error: 0.2144
Epoch 6/50
248/248 [==============================] - 67s 268ms/step - loss: 0.6559 - root_mean_squared_error: 0.1944 - val_loss: 0.6665

In [25]:
metadata_row.shape

(12,)

In [24]:
for x, label in train_dataset.take(1):
    print(x[0])

tf.Tensor(
[[[[3.03495347e-01 2.97099769e-01 2.66926020e-01]
   [5.57501733e-01 6.34595692e-01 5.04607379e-01]
   [7.55301297e-01 9.08299685e-01 6.80487573e-01]
   ...
   [7.13430047e-01 4.31835383e-01 4.14954633e-01]
   [6.37462020e-01 4.07729983e-01 3.72534871e-01]
   [7.32767582e-01 4.93317246e-01 4.61514473e-01]]

  [[1.71607226e-01 1.60770863e-01 1.25038683e-01]
   [2.12234527e-01 2.36350358e-01 1.68475434e-01]
   [3.93176079e-01 4.61035550e-01 3.56696874e-01]
   ...
   [7.35788405e-01 4.21619624e-01 4.19872463e-01]
   [6.95225954e-01 4.95910227e-01 4.55469280e-01]
   [6.59625828e-01 3.25985312e-01 3.30647558e-01]]

  [[5.93138039e-01 6.33406341e-01 5.81162453e-01]
   [1.59196317e-01 1.92710087e-01 1.50740951e-01]
   [1.52697727e-01 1.72056004e-01 1.40722141e-01]
   ...
   [7.80638695e-01 4.83607471e-01 4.85549152e-01]
   [7.26831436e-01 4.63939548e-01 4.53888506e-01]
   [6.27099395e-01 2.17629790e-01 2.56809264e-01]]

  ...

  [[2.88045377e-01 2.70702958e-01 1.91279694e-01]
   [3

2024-04-10 12:37:19.235572: W tensorflow/core/kernels/data/cache_dataset_ops.cc:858] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


In [7]:
images_train, images_test, metadata_train, metadata_test, y_train, y_test = train_test_split([1,2,3,4], [1,2,3,4], [1,2,3,4], test_size=0.5)

In [12]:
y_train

[1, 4]

In [13]:
y_test

[3, 2]

In [9]:
train_df.loc['0007de18844b0dbbb5e1f607da0606e0'][['Subject Focus', 'Eyes', 'Face', 'Near', 'Action', 'Accessory', 'Group',
       'Collage', 'Human', 'Occlusion', 'Info', 'Blur']].to_numpy()

array([0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0])

In [8]:
train_df.columns

Index(['Subject Focus', 'Eyes', 'Face', 'Near', 'Action', 'Accessory', 'Group',
       'Collage', 'Human', 'Occlusion', 'Info', 'Blur', 'Pawpularity'],
      dtype='object')